In [24]:
import numpy as np
import cv2
import tensorflow as tf
import serial
import requests
import time

In [25]:
# ser = serial.Serial("COM3", 9600)


In [26]:
face_model = cv2.CascadeClassifier('./haarcascades/haarcascade_frontalface_default.xml')

In [27]:
model = tf.keras.models.load_model("./masknet.h5")
# facerec = tf.keras.models.load_model("./facerec.h5")

In [ ]:
CLASSES = {
    0: "Please Remove Mask",
    1: "Good to go"
}

NAMES = {
    "Akshay": [],
    "Karma": [],
    "Luffy": [],
    "Priyanshu": [],
    "Riyesh": [],
    "Sangam": [],
    "Sangpo": [],
    "Sanji": [],
    "Shailaj": [],
    "Vibhrat": [],
     "Zoro": []
}

In [ ]:
def getSerialOutput():
    serialInst = serial.Serial()
    serialInst.baudrate = 9600
    serialInst.port = 'COM3'
    serialInst.open()

    while True:
        if serialInst.in_waiting:
            packet = serialInst.readline().decode('utf-8').rstrip()
            serial_data = list(map(float, packet.split(' ')))
            data = {
            "distance": serial_data[0],
            }
            return data

In [ ]:
vid  = cv2.VideoCapture(0)
start_time = time.time()
money = '0'
while True:
      ret, frame = vid.read()
      img = cv2.cvtColor(frame, cv2.IMREAD_GRAYSCALE)
      faces = face_model.detectMultiScale(img,scaleFactor=1.1, minNeighbors=4) #returns a list of (x,y,w,h) tuples
      dist = int(getSerialOutput()["distance"])
      
      for (x,y,w,h) in faces:
            if time.time() - start_time >= 10:
                  money = requests.get('http://localhost:5000/deduct').text
                  start_time = time.time()
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),1)
            label = [0 for i in range(len(faces))]
            new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #colored output image
            for i in range(len(faces)):
                  (x,y,w,h) = faces[i]
                  crop = new_img[y:y+h,x:x+w]
                  crop = cv2.resize(crop,(128,128))
                  crop = np.reshape(crop,[1,128,128,3])/255.0
                  mask_result = model.predict(crop)
                  class_index = np.argmax(mask_result[0])
                  result = CLASSES[class_index]
                  cv2.putText(img, str(round(mask_result[0][class_index], 2)), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 
                              1, (0, 0, 255), 2)
                  cv2.putText(img, result, (x+20,y+20), cv2.FONT_HERSHEY_SIMPLEX, 
                              1, (0, 0, 255), 2)

      cv2.putText(img, money, (20,50), cv2.FONT_HERSHEY_SIMPLEX, 
                              1, (0, 0, 255), 2)         
      cv2.putText(img, str(dist), (20,20), cv2.FONT_HERSHEY_SIMPLEX, 
                              1, (0, 0, 255), 2)
      if(dist>50):
            cv2.putText(img, "Payment done", (20,80), cv2.FONT_HERSHEY_SIMPLEX, 
                                    1, (0, 0, 255), 2)
      else:
            cv2.putText(img, "Please maintain some distance", (20,80), cv2.FONT_HERSHEY_SIMPLEX, 
                                    1, (0, 0, 255), 2)
      
      cv2.imshow("frame", img)
      if cv2.waitKey(1) & 0xFF == ord('q'):
        break

vid.release() 
cv2.destroyAllWindows()
